In [1]:
import os
from langchain_groq import ChatGroq
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())  


True

In [2]:
def get_chat_completion(prompt, model) ->str: 
    response = llm.invoke(prompt) 
    return response.content 

llm = ChatGroq(
    groq_api_key=os.getenv("GROQ_API_KEY"),
    model="llama-3.3-70b-versatile",
    temperature=0
)


In [3]:
template_string = """
You are an expert data analyst.
The user has a question about a plunger lift system, which has time series data for the following features:
1. Arrival Speed
2. Arrival Time Remaining
3. Casing Pressure
4. Current Non-Arrival Time
5. Down Hole Pressure
6. Line Pressure
7. Sales Meter Flow Rate
8. Sales Meter Static Pressure
9. Tubing Pressure

Each feature is stored in a separate CSV file (in directory ./La Vista 1H) with two columns: a 'timestamp' column and a column with the feature name.

Given the user's question, do the following:
1. Identify the relevant feature(s) from the list above. If none are found, respond with: **"Feature is not available."**
2. Identify any time constraints mentioned (e.g., specific date/time or time range).
3. Output a **Python Pandas code** that filters the CSV file(s) to get the requested information.

Output only the code, without any additional text or explanations.

User query: {query}
"""


In [4]:
from langchain.prompts import ChatPromptTemplate
prompt_template = ChatPromptTemplate.from_template(template_string) 

In [5]:
def extract_code_from_markdown(llm_output: str) -> str:
    if llm_output.startswith("```"):
        return "\n".join(
            line for line in llm_output.splitlines()
            if not line.strip().startswith("```")
        )
    return llm_output

user_query = "give me the latest 5 reading of arival speed"
message = prompt_template.format_messages(query=user_query)
resp = get_chat_completion(message, llm)
extracted_code = extract_code_from_markdown(resp)
extracted_code

"import pandas as pd\nimport glob\n\n# Load Arrival Speed data\narrival_speed_file = glob.glob('./La Vista 1H/Arrival Speed*.csv')[0]\narrival_speed_df = pd.read_csv(arrival_speed_file)\n\n# Ensure timestamp is in datetime format\narrival_speed_df['timestamp'] = pd.to_datetime(arrival_speed_df['timestamp'])\n\n# Sort by timestamp in descending order and get the latest 5 readings\nlatest_5_readings = arrival_speed_df.sort_values(by='timestamp', ascending=False).head(5)\n\nprint(latest_5_readings)"

In [6]:
exec(extracted_code)

                     timestamp    value
7200 2025-07-09 08:08:23+00:00  1057.80
7199 2025-07-09 08:06:23+00:00  1057.80
7198 2025-07-09 08:04:23+00:00  1057.80
7197 2025-07-09 08:02:23+00:00  1057.80
7196 2025-07-09 08:00:23+00:00  1122.95


In [ ]:
template_string_2 = """
Given below is a log of a plunger lift system.
Following are the events that can occur in a plunger lift system: 


[
  {
    "timestamp": "2025-07-18T15:00:00Z",
    "casing_pressure": 127.6,
    "tubing_pressure": 113.4,
    "line_pressure": 100.4,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 1
  },
  {
    "timestamp": "2025-07-18T15:01:00Z",
    "casing_pressure": 127.8,
    "tubing_pressure": 113.0,
    "line_pressure": 98.8,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 1
  },
  {
    "timestamp": "2025-07-18T15:02:00Z",
    "casing_pressure": 122.8,
    "tubing_pressure": 122.5,
    "line_pressure": 101.9,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 1
  },
  {
    "timestamp": "2025-07-18T15:03:00Z",
    "casing_pressure": 132.2,
    "tubing_pressure": 116.7,
    "line_pressure": 96.8,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 1
  },
  {
    "timestamp": "2025-07-18T15:04:00Z",
    "casing_pressure": 139.5,
    "tubing_pressure": 125.4,
    "line_pressure": 102.8,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 1
  },
  {
    "timestamp": "2025-07-18T15:05:00Z",
    "casing_pressure": 130.4,
    "tubing_pressure": 127.3,
    "line_pressure": 95.8,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 1
  },
  {
    "timestamp": "2025-07-18T15:06:00Z",
    "casing_pressure": 123.3,
    "tubing_pressure": 128.9,
    "line_pressure": 93.8,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 1
  },
  {
    "timestamp": "2025-07-18T15:07:00Z",
    "casing_pressure": 131.9,
    "tubing_pressure": 114.4,
    "line_pressure": 104.5,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 1
  },
  {
    "timestamp": "2025-07-18T15:08:00Z",
    "casing_pressure": 131.4,
    "tubing_pressure": 115.1,
    "line_pressure": 101.9,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 1
  },
  {
    "timestamp": "2025-07-18T15:09:00Z",
    "casing_pressure": 129.5,
    "tubing_pressure": 122.8,
    "line_pressure": 109.5,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 1
  },
  {
    "timestamp": "2025-07-18T15:10:00Z",
    "casing_pressure": 139.5,
    "tubing_pressure": 110.6,
    "line_pressure": 97.2,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 1
  },
  {
    "timestamp": "2025-07-18T15:11:00Z",
    "casing_pressure": 129.0,
    "tubing_pressure": 128.4,
    "line_pressure": 108.5,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 1
  },
  {
    "timestamp": "2025-07-18T15:12:00Z",
    "casing_pressure": 126.6,
    "tubing_pressure": 126.2,
    "line_pressure": 108.1,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 1
  },
  {
    "timestamp": "2025-07-18T15:13:00Z",
    "casing_pressure": 128.7,
    "tubing_pressure": 125.3,
    "line_pressure": 109.4,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 1
  },
  {
    "timestamp": "2025-07-18T15:14:00Z",
    "casing_pressure": 121.5,
    "tubing_pressure": 118.1,
    "line_pressure": 90.0,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 1
  },
  {
    "timestamp": "2025-07-18T15:15:00Z",
    "casing_pressure": 122.9,
    "tubing_pressure": 124.0,
    "line_pressure": 104.5,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 1
  },
  {
    "timestamp": "2025-07-18T15:16:00Z",
    "casing_pressure": 130.4,
    "tubing_pressure": 120.4,
    "line_pressure": 90.4,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 1
  },
  {
    "timestamp": "2025-07-18T15:17:00Z",
    "casing_pressure": 123.6,
    "tubing_pressure": 120.8,
    "line_pressure": 95.4,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 1
  },
  {
    "timestamp": "2025-07-18T15:18:00Z",
    "casing_pressure": 124.4,
    "tubing_pressure": 111.5,
    "line_pressure": 108.0,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 1
  },
  {
    "timestamp": "2025-07-18T15:19:00Z",
    "casing_pressure": 125.5,
    "tubing_pressure": 111.0,
    "line_pressure": 106.5,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 1
  },
  {
    "timestamp": "2025-07-18T15:20:00Z",
    "casing_pressure": 135.0,
    "tubing_pressure": 127.2,
    "line_pressure": 106.5,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 2
  },
  {
    "timestamp": "2025-07-18T15:21:00Z",
    "casing_pressure": 139.8,
    "tubing_pressure": 123.3,
    "line_pressure": 93.5,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 2
  },
  {
    "timestamp": "2025-07-18T15:22:00Z",
    "casing_pressure": 136.6,
    "tubing_pressure": 126.5,
    "line_pressure": 109.4,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 2
  },
  {
    "timestamp": "2025-07-18T15:23:00Z",
    "casing_pressure": 125.7,
    "tubing_pressure": 118.1,
    "line_pressure": 107.9,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 2
  },
  {
    "timestamp": "2025-07-18T15:24:00Z",
    "casing_pressure": 133.2,
    "tubing_pressure": 119.5,
    "line_pressure": 100.5,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 2
  },
  {
    "timestamp": "2025-07-18T15:25:00Z",
    "casing_pressure": 126.4,
    "tubing_pressure": 112.6,
    "line_pressure": 101.7,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 2
  },
  {
    "timestamp": "2025-07-18T15:26:00Z",
    "casing_pressure": 136.4,
    "tubing_pressure": 121.0,
    "line_pressure": 109.1,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 2
  },
  {
    "timestamp": "2025-07-18T15:27:00Z",
    "casing_pressure": 132.6,
    "tubing_pressure": 113.1,
    "line_pressure": 97.9,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 2
  },
  {
    "timestamp": "2025-07-18T15:28:00Z",
    "casing_pressure": 138.8,
    "tubing_pressure": 114.7,
    "line_pressure": 99.1,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 2
  },
  {
    "timestamp": "2025-07-18T15:29:00Z",
    "casing_pressure": 131.0,
    "tubing_pressure": 128.1,
    "line_pressure": 103.7,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 2
  },
  {
    "timestamp": "2025-07-18T15:30:00Z",
    "casing_pressure": 137.7,
    "tubing_pressure": 129.6,
    "line_pressure": 108.1,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 2
  },
  {
    "timestamp": "2025-07-18T15:31:00Z",
    "casing_pressure": 131.2,
    "tubing_pressure": 110.2,
    "line_pressure": 94.6,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 2
  },
  {
    "timestamp": "2025-07-18T15:32:00Z",
    "casing_pressure": 133.5,
    "tubing_pressure": 122.8,
    "line_pressure": 103.0,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 2
  },
  {
    "timestamp": "2025-07-18T15:33:00Z",
    "casing_pressure": 139.6,
    "tubing_pressure": 124.3,
    "line_pressure": 105.4,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 2
  },
  {
    "timestamp": "2025-07-18T15:34:00Z",
    "casing_pressure": 120.5,
    "tubing_pressure": 115.3,
    "line_pressure": 104.2,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 2
  },
  {
    "timestamp": "2025-07-18T15:35:00Z",
    "casing_pressure": 139.4,
    "tubing_pressure": 110.2,
    "line_pressure": 99.8,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 2
  },
  {
    "timestamp": "2025-07-18T15:36:00Z",
    "casing_pressure": 133.7,
    "tubing_pressure": 129.3,
    "line_pressure": 99.4,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 2
  },
  {
    "timestamp": "2025-07-18T15:37:00Z",
    "casing_pressure": 121.9,
    "tubing_pressure": 129.5,
    "line_pressure": 103.5,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 2
  },
  {
    "timestamp": "2025-07-18T15:38:00Z",
    "casing_pressure": 124.7,
    "tubing_pressure": 118.9,
    "line_pressure": 90.7,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 2
  },
  {
    "timestamp": "2025-07-18T15:39:00Z",
    "casing_pressure": 129.8,
    "tubing_pressure": 129.2,
    "line_pressure": 104.0,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 2
  },
  {
    "timestamp": "2025-07-18T15:40:00Z",
    "casing_pressure": 127.0,
    "tubing_pressure": 119.8,
    "line_pressure": 99.1,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 3
  },
  {
    "timestamp": "2025-07-18T15:41:00Z",
    "casing_pressure": 138.2,
    "tubing_pressure": 116.7,
    "line_pressure": 94.3,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 3
  },
  {
    "timestamp": "2025-07-18T15:42:00Z",
    "casing_pressure": 139.5,
    "tubing_pressure": 125.8,
    "line_pressure": 95.3,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 3
  },
  {
    "timestamp": "2025-07-18T15:43:00Z",
    "casing_pressure": 128.1,
    "tubing_pressure": 117.1,
    "line_pressure": 98.4,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 3
  },
  {
    "timestamp": "2025-07-18T15:44:00Z",
    "casing_pressure": 121.5,
    "tubing_pressure": 111.5,
    "line_pressure": 100.5,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 3
  },
  {
    "timestamp": "2025-07-18T15:45:00Z",
    "casing_pressure": 137.1,
    "tubing_pressure": 128.4,
    "line_pressure": 101.9,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 3
  },
  {
    "timestamp": "2025-07-18T15:46:00Z",
    "casing_pressure": 139.7,
    "tubing_pressure": 118.7,
    "line_pressure": 99.8,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 3
  },
  {
    "timestamp": "2025-07-18T15:47:00Z",
    "casing_pressure": 127.6,
    "tubing_pressure": 129.0,
    "line_pressure": 90.3,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 3
  },
  {
    "timestamp": "2025-07-18T15:48:00Z",
    "casing_pressure": 120.9,
    "tubing_pressure": 114.7,
    "line_pressure": 93.9,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 3
  },
  {
    "timestamp": "2025-07-18T15:49:00Z",
    "casing_pressure": 130.8,
    "tubing_pressure": 124.4,
    "line_pressure": 100.5,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 3
  },
  {
    "timestamp": "2025-07-18T15:50:00Z",
    "casing_pressure": 138.5,
    "tubing_pressure": 112.1,
    "line_pressure": 101.3,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 3
  },
  {
    "timestamp": "2025-07-18T15:51:00Z",
    "casing_pressure": 131.8,
    "tubing_pressure": 113.4,
    "line_pressure": 90.6,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 3
  },
  {
    "timestamp": "2025-07-18T15:52:00Z",
    "casing_pressure": 136.1,
    "tubing_pressure": 117.6,
    "line_pressure": 94.8,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 3
  },
  {
    "timestamp": "2025-07-18T15:53:00Z",
    "casing_pressure": 124.1,
    "tubing_pressure": 129.2,
    "line_pressure": 93.6,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 3
  },
  {
    "timestamp": "2025-07-18T15:54:00Z",
    "casing_pressure": 120.9,
    "tubing_pressure": 111.7,
    "line_pressure": 90.1,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 3
  },
  {
    "timestamp": "2025-07-18T15:55:00Z",
    "casing_pressure": 137.7,
    "tubing_pressure": 126.0,
    "line_pressure": 98.9,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 3
  },
  {
    "timestamp": "2025-07-18T15:56:00Z",
    "casing_pressure": 126.9,
    "tubing_pressure": 130.0,
    "line_pressure": 94.8,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 3
  },
  {
    "timestamp": "2025-07-18T15:57:00Z",
    "casing_pressure": 124.2,
    "tubing_pressure": 126.9,
    "line_pressure": 106.0,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 3
  },
  {
    "timestamp": "2025-07-18T15:58:00Z",
    "casing_pressure": 121.7,
    "tubing_pressure": 123.3,
    "line_pressure": 105.4,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 3
  },
  {
    "timestamp": "2025-07-18T15:59:00Z",
    "casing_pressure": 127.0,
    "tubing_pressure": 126.1,
    "line_pressure": 94.0,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 3
  },
  {
    "timestamp": "2025-07-18T16:00:00Z",
    "casing_pressure": 84.9,
    "tubing_pressure": 71.0,
    "line_pressure": 62.2,
    "non_arrival_count": 1,
    "well_id": "WELL-001",
    "cycle_id": 4
  },
  {
    "timestamp": "2025-07-18T16:01:00Z",
    "casing_pressure": 89.4,
    "tubing_pressure": 73.0,
    "line_pressure": 70.6,
    "non_arrival_count": 1,
    "well_id": "WELL-001",
    "cycle_id": 4
  },
  {
    "timestamp": "2025-07-18T16:02:00Z",
    "casing_pressure": 86.3,
    "tubing_pressure": 77.8,
    "line_pressure": 74.0,
    "non_arrival_count": 1,
    "well_id": "WELL-001",
    "cycle_id": 4
  },
  {
    "timestamp": "2025-07-18T16:03:00Z",
    "casing_pressure": 91.9,
    "tubing_pressure": 73.4,
    "line_pressure": 71.9,
    "non_arrival_count": 1,
    "well_id": "WELL-001",
    "cycle_id": 4
  },
  {
    "timestamp": "2025-07-18T16:04:00Z",
    "casing_pressure": 86.5,
    "tubing_pressure": 84.0,
    "line_pressure": 65.9,
    "non_arrival_count": 1,
    "well_id": "WELL-001",
    "cycle_id": 4
  },
  {
    "timestamp": "2025-07-18T16:05:00Z",
    "casing_pressure": 89.2,
    "tubing_pressure": 77.0,
    "line_pressure": 61.1,
    "non_arrival_count": 1,
    "well_id": "WELL-001",
    "cycle_id": 4
  },
  {
    "timestamp": "2025-07-18T16:06:00Z",
    "casing_pressure": 88.2,
    "tubing_pressure": 78.8,
    "line_pressure": 66.9,
    "non_arrival_count": 1,
    "well_id": "WELL-001",
    "cycle_id": 4
  },
  {
    "timestamp": "2025-07-18T16:07:00Z",
    "casing_pressure": 85.6,
    "tubing_pressure": 79.4,
    "line_pressure": 67.1,
    "non_arrival_count": 1,
    "well_id": "WELL-001",
    "cycle_id": 4
  },
  {
    "timestamp": "2025-07-18T16:08:00Z",
    "casing_pressure": 88.1,
    "tubing_pressure": 71.5,
    "line_pressure": 70.3,
    "non_arrival_count": 1,
    "well_id": "WELL-001",
    "cycle_id": 4
  },
  {
    "timestamp": "2025-07-18T16:09:00Z",
    "casing_pressure": 87.5,
    "tubing_pressure": 79.4,
    "line_pressure": 70.0,
    "non_arrival_count": 1,
    "well_id": "WELL-001",
    "cycle_id": 4
  },
  {
    "timestamp": "2025-07-18T16:10:00Z",
    "casing_pressure": 82.3,
    "tubing_pressure": 81.4,
    "line_pressure": 71.9,
    "non_arrival_count": 1,
    "well_id": "WELL-001",
    "cycle_id": 4
  },
  {
    "timestamp": "2025-07-18T16:11:00Z",
    "casing_pressure": 82.2,
    "tubing_pressure": 75.8,
    "line_pressure": 71.7,
    "non_arrival_count": 1,
    "well_id": "WELL-001",
    "cycle_id": 4
  },
  {
    "timestamp": "2025-07-18T16:12:00Z",
    "casing_pressure": 81.4,
    "tubing_pressure": 84.1,
    "line_pressure": 64.3,
    "non_arrival_count": 1,
    "well_id": "WELL-001",
    "cycle_id": 4
  },
  {
    "timestamp": "2025-07-18T16:13:00Z",
    "casing_pressure": 85.4,
    "tubing_pressure": 73.7,
    "line_pressure": 61.2,
    "non_arrival_count": 1,
    "well_id": "WELL-001",
    "cycle_id": 4
  },
  {
    "timestamp": "2025-07-18T16:14:00Z",
    "casing_pressure": 85.4,
    "tubing_pressure": 77.3,
    "line_pressure": 65.3,
    "non_arrival_count": 1,
    "well_id": "WELL-001",
    "cycle_id": 4
  },
  {
    "timestamp": "2025-07-18T16:15:00Z",
    "casing_pressure": 89.6,
    "tubing_pressure": 75.9,
    "line_pressure": 67.9,
    "non_arrival_count": 1,
    "well_id": "WELL-001",
    "cycle_id": 4
  },
  {
    "timestamp": "2025-07-18T16:16:00Z",
    "casing_pressure": 87.8,
    "tubing_pressure": 82.0,
    "line_pressure": 72.6,
    "non_arrival_count": 1,
    "well_id": "WELL-001",
    "cycle_id": 4
  },
  {
    "timestamp": "2025-07-18T16:17:00Z",
    "casing_pressure": 88.4,
    "tubing_pressure": 82.8,
    "line_pressure": 74.8,
    "non_arrival_count": 1,
    "well_id": "WELL-001",
    "cycle_id": 4
  },
  {
    "timestamp": "2025-07-18T16:18:00Z",
    "casing_pressure": 92.0,
    "tubing_pressure": 78.8,
    "line_pressure": 61.1,
    "non_arrival_count": 1,
    "well_id": "WELL-001",
    "cycle_id": 4
  },
  {
    "timestamp": "2025-07-18T16:19:00Z",
    "casing_pressure": 92.9,
    "tubing_pressure": 81.2,
    "line_pressure": 69.3,
    "non_arrival_count": 1,
    "well_id": "WELL-001",
    "cycle_id": 4
  },
  {
    "timestamp": "2025-07-18T16:20:00Z",
    "casing_pressure": 125.4,
    "tubing_pressure": 110.5,
    "line_pressure": 101.6,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 5
  },
  {
    "timestamp": "2025-07-18T16:21:00Z",
    "casing_pressure": 135.2,
    "tubing_pressure": 117.4,
    "line_pressure": 106.3,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 5
  },
  {
    "timestamp": "2025-07-18T16:22:00Z",
    "casing_pressure": 138.2,
    "tubing_pressure": 119.5,
    "line_pressure": 96.4,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 5
  },
  {
    "timestamp": "2025-07-18T16:23:00Z",
    "casing_pressure": 130.8,
    "tubing_pressure": 122.2,
    "line_pressure": 94.6,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 5
  },
  {
    "timestamp": "2025-07-18T16:24:00Z",
    "casing_pressure": 136.4,
    "tubing_pressure": 128.0,
    "line_pressure": 101.7,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 5
  },
  {
    "timestamp": "2025-07-18T16:25:00Z",
    "casing_pressure": 135.3,
    "tubing_pressure": 122.9,
    "line_pressure": 97.5,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 5
  },
  {
    "timestamp": "2025-07-18T16:26:00Z",
    "casing_pressure": 131.0,
    "tubing_pressure": 120.9,
    "line_pressure": 107.1,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 5
  },
  {
    "timestamp": "2025-07-18T16:27:00Z",
    "casing_pressure": 135.8,
    "tubing_pressure": 110.4,
    "line_pressure": 105.6,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 5
  },
  {
    "timestamp": "2025-07-18T16:28:00Z",
    "casing_pressure": 134.3,
    "tubing_pressure": 117.3,
    "line_pressure": 102.4,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 5
  },
  {
    "timestamp": "2025-07-18T16:29:00Z",
    "casing_pressure": 123.2,
    "tubing_pressure": 118.9,
    "line_pressure": 105.9,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 5
  },
  {
    "timestamp": "2025-07-18T16:30:00Z",
    "casing_pressure": 121.5,
    "tubing_pressure": 113.8,
    "line_pressure": 98.3,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 5
  },
  {
    "timestamp": "2025-07-18T16:31:00Z",
    "casing_pressure": 129.4,
    "tubing_pressure": 122.7,
    "line_pressure": 97.3,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 5
  },
  {
    "timestamp": "2025-07-18T16:32:00Z",
    "casing_pressure": 127.5,
    "tubing_pressure": 124.8,
    "line_pressure": 97.3,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 5
  },
  {
    "timestamp": "2025-07-18T16:33:00Z",
    "casing_pressure": 124.7,
    "tubing_pressure": 125.7,
    "line_pressure": 102.3,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 5
  },
  {
    "timestamp": "2025-07-18T16:34:00Z",
    "casing_pressure": 123.3,
    "tubing_pressure": 121.3,
    "line_pressure": 97.3,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 5
  },
  {
    "timestamp": "2025-07-18T16:35:00Z",
    "casing_pressure": 120.3,
    "tubing_pressure": 119.9,
    "line_pressure": 97.4,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 5
  },
  {
    "timestamp": "2025-07-18T16:36:00Z",
    "casing_pressure": 122.0,
    "tubing_pressure": 116.9,
    "line_pressure": 102.0,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 5
  },
  {
    "timestamp": "2025-07-18T16:37:00Z",
    "casing_pressure": 127.9,
    "tubing_pressure": 125.8,
    "line_pressure": 99.8,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 5
  },
  {
    "timestamp": "2025-07-18T16:38:00Z",
    "casing_pressure": 121.0,
    "tubing_pressure": 115.6,
    "line_pressure": 103.4,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 5
  },
  {
    "timestamp": "2025-07-18T16:39:00Z",
    "casing_pressure": 126.9,
    "tubing_pressure": 129.3,
    "line_pressure": 107.8,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 5
  }
]





"""

In [ ]:
prompt_string = """
You are a data-savvy assistant that ingests plunger-lift cycle data.
Below are concise behavioral definitions for every variable that appears in the five basic event labels.
Each definition tells you exactly how the variable behaves (or misbehaves) when the corresponding event is true.
Use these signatures to answer questions, classify cycles, or generate explanations.
— Event: NonArrivalEvent —
Meaning: plunger never reaches surface.

    tm → timer counts upward until max arrival window, then freezes at null / timeout; no arrival spike.
    arrival → remains FALSE for entire cycle.
    expected_arrival_window → static numeric bound, unchanged.
    plunger_position (if available) → flattens at intermediate depth, above bumper spring, below surface.
    shutdown_triggered_by → transitions to “timeout” once window expires.

— Event: UnsafeVelocityEvent —
Meaning: plunger arrives with unsafe speed.

    vr → spikes above safe_velocity_max or drops below safe_velocity_min at arrival instant.
    safe_velocity_min / safe_velocity_max → fixed thresholds, constant throughout cycle.

— Event: LowRewardEvent —
Meaning: cycle reward score below economic threshold.

    Eff → plateaus or decays, never meets reward criteria.
    gas_volume → flattens or declines, insufficient incremental gas.
    tc, Cg, STBPD → actual vs target diverge, dragging reward_score down.
    reward_score → stays continuously below reward_threshold; slope flat or negative.
    tuning_window (shut-in / after-flow) → static or oscillates without lifting reward.

— Event: ShortShutInEvent —
Meaning: shut-in ends before adequate casing pressure recovery.

    ts → plateau stops prematurely, value < recommended_min_shut_in.
    casing_pressure_trend → slope flattens early, bends over before recovery target.
    pressure_recovered → remains FALSE entire cycle; ΔP target never reached.

— Event: ExcessAfterflowEvent —
Meaning: after-flow continues longer than optimal, wasting gas.

    ta → continues rising past optimal_afterflow_duration, plateaus at excessive value.
    Eff → flattens or declines; no more liquid removed.
    R_l → downward slope, slug depleting.
    FR → falling curve, gas fraction in slug decreases.
    line_pressure_drop → continues decreasing, gas energy diminishing.
    early_liquid_breakthrough → may switch to TRUE as liquid re-enters tubing.

Use these behavioral signatures to answer any subsequent questions.


Here are the log of the system: 
[
  {
    "timestamp": "2025-07-18T15:00:00Z",
    "casing_pressure": 127.6,
    "tubing_pressure": 113.4,
    "line_pressure": 100.4,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 1
  },
  {
    "timestamp": "2025-07-18T15:01:00Z",
    "casing_pressure": 127.8,
    "tubing_pressure": 113.0,
    "line_pressure": 98.8,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 1
  },
  {
    "timestamp": "2025-07-18T15:02:00Z",
    "casing_pressure": 122.8,
    "tubing_pressure": 122.5,
    "line_pressure": 101.9,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 1
  },
  {
    "timestamp": "2025-07-18T15:03:00Z",
    "casing_pressure": 132.2,
    "tubing_pressure": 116.7,
    "line_pressure": 96.8,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 1
  },
  {
    "timestamp": "2025-07-18T15:04:00Z",
    "casing_pressure": 139.5,
    "tubing_pressure": 125.4,
    "line_pressure": 102.8,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 1
  },
  {
    "timestamp": "2025-07-18T15:05:00Z",
    "casing_pressure": 130.4,
    "tubing_pressure": 127.3,
    "line_pressure": 95.8,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 1
  },
  {
    "timestamp": "2025-07-18T15:06:00Z",
    "casing_pressure": 123.3,
    "tubing_pressure": 128.9,
    "line_pressure": 93.8,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 1
  },
  {
    "timestamp": "2025-07-18T15:07:00Z",
    "casing_pressure": 131.9,
    "tubing_pressure": 114.4,
    "line_pressure": 104.5,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 1
  },
  {
    "timestamp": "2025-07-18T15:08:00Z",
    "casing_pressure": 131.4,
    "tubing_pressure": 115.1,
    "line_pressure": 101.9,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 1
  },
  {
    "timestamp": "2025-07-18T15:09:00Z",
    "casing_pressure": 129.5,
    "tubing_pressure": 122.8,
    "line_pressure": 109.5,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 1
  },
  {
    "timestamp": "2025-07-18T15:10:00Z",
    "casing_pressure": 139.5,
    "tubing_pressure": 110.6,
    "line_pressure": 97.2,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 1
  },
  {
    "timestamp": "2025-07-18T15:11:00Z",
    "casing_pressure": 129.0,
    "tubing_pressure": 128.4,
    "line_pressure": 108.5,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 1
  },
  {
    "timestamp": "2025-07-18T15:12:00Z",
    "casing_pressure": 126.6,
    "tubing_pressure": 126.2,
    "line_pressure": 108.1,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 1
  },
  {
    "timestamp": "2025-07-18T15:13:00Z",
    "casing_pressure": 128.7,
    "tubing_pressure": 125.3,
    "line_pressure": 109.4,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 1
  },
  {
    "timestamp": "2025-07-18T15:14:00Z",
    "casing_pressure": 121.5,
    "tubing_pressure": 118.1,
    "line_pressure": 90.0,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 1
  },
  {
    "timestamp": "2025-07-18T15:15:00Z",
    "casing_pressure": 122.9,
    "tubing_pressure": 124.0,
    "line_pressure": 104.5,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 1
  },
  {
    "timestamp": "2025-07-18T15:16:00Z",
    "casing_pressure": 130.4,
    "tubing_pressure": 120.4,
    "line_pressure": 90.4,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 1
  },
  {
    "timestamp": "2025-07-18T15:17:00Z",
    "casing_pressure": 123.6,
    "tubing_pressure": 120.8,
    "line_pressure": 95.4,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 1
  },
  {
    "timestamp": "2025-07-18T15:18:00Z",
    "casing_pressure": 124.4,
    "tubing_pressure": 111.5,
    "line_pressure": 108.0,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 1
  },
  {
    "timestamp": "2025-07-18T15:19:00Z",
    "casing_pressure": 125.5,
    "tubing_pressure": 111.0,
    "line_pressure": 106.5,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 1
  },
  {
    "timestamp": "2025-07-18T15:20:00Z",
    "casing_pressure": 135.0,
    "tubing_pressure": 127.2,
    "line_pressure": 106.5,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 2
  },
  {
    "timestamp": "2025-07-18T15:21:00Z",
    "casing_pressure": 139.8,
    "tubing_pressure": 123.3,
    "line_pressure": 93.5,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 2
  },
  {
    "timestamp": "2025-07-18T15:22:00Z",
    "casing_pressure": 136.6,
    "tubing_pressure": 126.5,
    "line_pressure": 109.4,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 2
  },
  {
    "timestamp": "2025-07-18T15:23:00Z",
    "casing_pressure": 125.7,
    "tubing_pressure": 118.1,
    "line_pressure": 107.9,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 2
  },
  {
    "timestamp": "2025-07-18T15:24:00Z",
    "casing_pressure": 133.2,
    "tubing_pressure": 119.5,
    "line_pressure": 100.5,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 2
  },
  {
    "timestamp": "2025-07-18T15:25:00Z",
    "casing_pressure": 126.4,
    "tubing_pressure": 112.6,
    "line_pressure": 101.7,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 2
  },
  {
    "timestamp": "2025-07-18T15:26:00Z",
    "casing_pressure": 136.4,
    "tubing_pressure": 121.0,
    "line_pressure": 109.1,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 2
  },
  {
    "timestamp": "2025-07-18T15:27:00Z",
    "casing_pressure": 132.6,
    "tubing_pressure": 113.1,
    "line_pressure": 97.9,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 2
  },
  {
    "timestamp": "2025-07-18T15:28:00Z",
    "casing_pressure": 138.8,
    "tubing_pressure": 114.7,
    "line_pressure": 99.1,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 2
  },
  {
    "timestamp": "2025-07-18T15:29:00Z",
    "casing_pressure": 131.0,
    "tubing_pressure": 128.1,
    "line_pressure": 103.7,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 2
  },
  {
    "timestamp": "2025-07-18T15:30:00Z",
    "casing_pressure": 137.7,
    "tubing_pressure": 129.6,
    "line_pressure": 108.1,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 2
  },
  {
    "timestamp": "2025-07-18T15:31:00Z",
    "casing_pressure": 131.2,
    "tubing_pressure": 110.2,
    "line_pressure": 94.6,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 2
  },
  {
    "timestamp": "2025-07-18T15:32:00Z",
    "casing_pressure": 133.5,
    "tubing_pressure": 122.8,
    "line_pressure": 103.0,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 2
  },
  {
    "timestamp": "2025-07-18T15:33:00Z",
    "casing_pressure": 139.6,
    "tubing_pressure": 124.3,
    "line_pressure": 105.4,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 2
  },
  {
    "timestamp": "2025-07-18T15:34:00Z",
    "casing_pressure": 120.5,
    "tubing_pressure": 115.3,
    "line_pressure": 104.2,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 2
  },
  {
    "timestamp": "2025-07-18T15:35:00Z",
    "casing_pressure": 139.4,
    "tubing_pressure": 110.2,
    "line_pressure": 99.8,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 2
  },
  {
    "timestamp": "2025-07-18T15:36:00Z",
    "casing_pressure": 133.7,
    "tubing_pressure": 129.3,
    "line_pressure": 99.4,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 2
  },
  {
    "timestamp": "2025-07-18T15:37:00Z",
    "casing_pressure": 121.9,
    "tubing_pressure": 129.5,
    "line_pressure": 103.5,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 2
  },
  {
    "timestamp": "2025-07-18T15:38:00Z",
    "casing_pressure": 124.7,
    "tubing_pressure": 118.9,
    "line_pressure": 90.7,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 2
  },
  {
    "timestamp": "2025-07-18T15:39:00Z",
    "casing_pressure": 129.8,
    "tubing_pressure": 129.2,
    "line_pressure": 104.0,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 2
  },
  {
    "timestamp": "2025-07-18T15:40:00Z",
    "casing_pressure": 127.0,
    "tubing_pressure": 119.8,
    "line_pressure": 99.1,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 3
  },
  {
    "timestamp": "2025-07-18T15:41:00Z",
    "casing_pressure": 138.2,
    "tubing_pressure": 116.7,
    "line_pressure": 94.3,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 3
  },
  {
    "timestamp": "2025-07-18T15:42:00Z",
    "casing_pressure": 139.5,
    "tubing_pressure": 125.8,
    "line_pressure": 95.3,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 3
  },
  {
    "timestamp": "2025-07-18T15:43:00Z",
    "casing_pressure": 128.1,
    "tubing_pressure": 117.1,
    "line_pressure": 98.4,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 3
  },
  {
    "timestamp": "2025-07-18T15:44:00Z",
    "casing_pressure": 121.5,
    "tubing_pressure": 111.5,
    "line_pressure": 100.5,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 3
  },
  {
    "timestamp": "2025-07-18T15:45:00Z",
    "casing_pressure": 137.1,
    "tubing_pressure": 128.4,
    "line_pressure": 101.9,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 3
  },
  {
    "timestamp": "2025-07-18T15:46:00Z",
    "casing_pressure": 139.7,
    "tubing_pressure": 118.7,
    "line_pressure": 99.8,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 3
  },
  {
    "timestamp": "2025-07-18T15:47:00Z",
    "casing_pressure": 127.6,
    "tubing_pressure": 129.0,
    "line_pressure": 90.3,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 3
  },
  {
    "timestamp": "2025-07-18T15:48:00Z",
    "casing_pressure": 120.9,
    "tubing_pressure": 114.7,
    "line_pressure": 93.9,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 3
  },
  {
    "timestamp": "2025-07-18T15:49:00Z",
    "casing_pressure": 130.8,
    "tubing_pressure": 124.4,
    "line_pressure": 100.5,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 3
  },
  {
    "timestamp": "2025-07-18T15:50:00Z",
    "casing_pressure": 138.5,
    "tubing_pressure": 112.1,
    "line_pressure": 101.3,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 3
  },
  {
    "timestamp": "2025-07-18T15:51:00Z",
    "casing_pressure": 131.8,
    "tubing_pressure": 113.4,
    "line_pressure": 90.6,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 3
  },
  {
    "timestamp": "2025-07-18T15:52:00Z",
    "casing_pressure": 136.1,
    "tubing_pressure": 117.6,
    "line_pressure": 94.8,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 3
  },
  {
    "timestamp": "2025-07-18T15:53:00Z",
    "casing_pressure": 124.1,
    "tubing_pressure": 129.2,
    "line_pressure": 93.6,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 3
  },
  {
    "timestamp": "2025-07-18T15:54:00Z",
    "casing_pressure": 120.9,
    "tubing_pressure": 111.7,
    "line_pressure": 90.1,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 3
  },
  {
    "timestamp": "2025-07-18T15:55:00Z",
    "casing_pressure": 137.7,
    "tubing_pressure": 126.0,
    "line_pressure": 98.9,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 3
  },
  {
    "timestamp": "2025-07-18T15:56:00Z",
    "casing_pressure": 126.9,
    "tubing_pressure": 130.0,
    "line_pressure": 94.8,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 3
  },
  {
    "timestamp": "2025-07-18T15:57:00Z",
    "casing_pressure": 124.2,
    "tubing_pressure": 126.9,
    "line_pressure": 106.0,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 3
  },
  {
    "timestamp": "2025-07-18T15:58:00Z",
    "casing_pressure": 121.7,
    "tubing_pressure": 123.3,
    "line_pressure": 105.4,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 3
  },
  {
    "timestamp": "2025-07-18T15:59:00Z",
    "casing_pressure": 127.0,
    "tubing_pressure": 126.1,
    "line_pressure": 94.0,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 3
  },
  {
    "timestamp": "2025-07-18T16:00:00Z",
    "casing_pressure": 84.9,
    "tubing_pressure": 71.0,
    "line_pressure": 62.2,
    "non_arrival_count": 1,
    "well_id": "WELL-001",
    "cycle_id": 4
  },
  {
    "timestamp": "2025-07-18T16:01:00Z",
    "casing_pressure": 89.4,
    "tubing_pressure": 73.0,
    "line_pressure": 70.6,
    "non_arrival_count": 1,
    "well_id": "WELL-001",
    "cycle_id": 4
  },
  {
    "timestamp": "2025-07-18T16:02:00Z",
    "casing_pressure": 86.3,
    "tubing_pressure": 77.8,
    "line_pressure": 74.0,
    "non_arrival_count": 1,
    "well_id": "WELL-001",
    "cycle_id": 4
  },
  {
    "timestamp": "2025-07-18T16:03:00Z",
    "casing_pressure": 91.9,
    "tubing_pressure": 73.4,
    "line_pressure": 71.9,
    "non_arrival_count": 1,
    "well_id": "WELL-001",
    "cycle_id": 4
  },
  {
    "timestamp": "2025-07-18T16:04:00Z",
    "casing_pressure": 86.5,
    "tubing_pressure": 84.0,
    "line_pressure": 65.9,
    "non_arrival_count": 1,
    "well_id": "WELL-001",
    "cycle_id": 4
  },
  {
    "timestamp": "2025-07-18T16:05:00Z",
    "casing_pressure": 89.2,
    "tubing_pressure": 77.0,
    "line_pressure": 61.1,
    "non_arrival_count": 1,
    "well_id": "WELL-001",
    "cycle_id": 4
  },
  {
    "timestamp": "2025-07-18T16:06:00Z",
    "casing_pressure": 88.2,
    "tubing_pressure": 78.8,
    "line_pressure": 66.9,
    "non_arrival_count": 1,
    "well_id": "WELL-001",
    "cycle_id": 4
  },
  {
    "timestamp": "2025-07-18T16:07:00Z",
    "casing_pressure": 85.6,
    "tubing_pressure": 79.4,
    "line_pressure": 67.1,
    "non_arrival_count": 1,
    "well_id": "WELL-001",
    "cycle_id": 4
  },
  {
    "timestamp": "2025-07-18T16:08:00Z",
    "casing_pressure": 88.1,
    "tubing_pressure": 71.5,
    "line_pressure": 70.3,
    "non_arrival_count": 1,
    "well_id": "WELL-001",
    "cycle_id": 4
  },
  {
    "timestamp": "2025-07-18T16:09:00Z",
    "casing_pressure": 87.5,
    "tubing_pressure": 79.4,
    "line_pressure": 70.0,
    "non_arrival_count": 1,
    "well_id": "WELL-001",
    "cycle_id": 4
  },
  {
    "timestamp": "2025-07-18T16:10:00Z",
    "casing_pressure": 82.3,
    "tubing_pressure": 81.4,
    "line_pressure": 71.9,
    "non_arrival_count": 1,
    "well_id": "WELL-001",
    "cycle_id": 4
  },
  {
    "timestamp": "2025-07-18T16:11:00Z",
    "casing_pressure": 82.2,
    "tubing_pressure": 75.8,
    "line_pressure": 71.7,
    "non_arrival_count": 1,
    "well_id": "WELL-001",
    "cycle_id": 4
  },
  {
    "timestamp": "2025-07-18T16:12:00Z",
    "casing_pressure": 81.4,
    "tubing_pressure": 84.1,
    "line_pressure": 64.3,
    "non_arrival_count": 1,
    "well_id": "WELL-001",
    "cycle_id": 4
  },
  {
    "timestamp": "2025-07-18T16:13:00Z",
    "casing_pressure": 85.4,
    "tubing_pressure": 73.7,
    "line_pressure": 61.2,
    "non_arrival_count": 1,
    "well_id": "WELL-001",
    "cycle_id": 4
  },
  {
    "timestamp": "2025-07-18T16:14:00Z",
    "casing_pressure": 85.4,
    "tubing_pressure": 77.3,
    "line_pressure": 65.3,
    "non_arrival_count": 1,
    "well_id": "WELL-001",
    "cycle_id": 4
  },
  {
    "timestamp": "2025-07-18T16:15:00Z",
    "casing_pressure": 89.6,
    "tubing_pressure": 75.9,
    "line_pressure": 67.9,
    "non_arrival_count": 1,
    "well_id": "WELL-001",
    "cycle_id": 4
  },
  {
    "timestamp": "2025-07-18T16:16:00Z",
    "casing_pressure": 87.8,
    "tubing_pressure": 82.0,
    "line_pressure": 72.6,
    "non_arrival_count": 1,
    "well_id": "WELL-001",
    "cycle_id": 4
  },
  {
    "timestamp": "2025-07-18T16:17:00Z",
    "casing_pressure": 88.4,
    "tubing_pressure": 82.8,
    "line_pressure": 74.8,
    "non_arrival_count": 1,
    "well_id": "WELL-001",
    "cycle_id": 4
  },
  {
    "timestamp": "2025-07-18T16:18:00Z",
    "casing_pressure": 92.0,
    "tubing_pressure": 78.8,
    "line_pressure": 61.1,
    "non_arrival_count": 1,
    "well_id": "WELL-001",
    "cycle_id": 4
  },
  {
    "timestamp": "2025-07-18T16:19:00Z",
    "casing_pressure": 92.9,
    "tubing_pressure": 81.2,
    "line_pressure": 69.3,
    "non_arrival_count": 1,
    "well_id": "WELL-001",
    "cycle_id": 4
  },
  {
    "timestamp": "2025-07-18T16:20:00Z",
    "casing_pressure": 125.4,
    "tubing_pressure": 110.5,
    "line_pressure": 101.6,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 5
  },
  {
    "timestamp": "2025-07-18T16:21:00Z",
    "casing_pressure": 135.2,
    "tubing_pressure": 117.4,
    "line_pressure": 106.3,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 5
  },
  {
    "timestamp": "2025-07-18T16:22:00Z",
    "casing_pressure": 138.2,
    "tubing_pressure": 119.5,
    "line_pressure": 96.4,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 5
  },
  {
    "timestamp": "2025-07-18T16:23:00Z",
    "casing_pressure": 130.8,
    "tubing_pressure": 122.2,
    "line_pressure": 94.6,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 5
  },
  {
    "timestamp": "2025-07-18T16:24:00Z",
    "casing_pressure": 136.4,
    "tubing_pressure": 128.0,
    "line_pressure": 101.7,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 5
  },
  {
    "timestamp": "2025-07-18T16:25:00Z",
    "casing_pressure": 135.3,
    "tubing_pressure": 122.9,
    "line_pressure": 97.5,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 5
  },
  {
    "timestamp": "2025-07-18T16:26:00Z",
    "casing_pressure": 131.0,
    "tubing_pressure": 120.9,
    "line_pressure": 107.1,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 5
  },
  {
    "timestamp": "2025-07-18T16:27:00Z",
    "casing_pressure": 135.8,
    "tubing_pressure": 110.4,
    "line_pressure": 105.6,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 5
  },
  {
    "timestamp": "2025-07-18T16:28:00Z",
    "casing_pressure": 134.3,
    "tubing_pressure": 117.3,
    "line_pressure": 102.4,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 5
  },
  {
    "timestamp": "2025-07-18T16:29:00Z",
    "casing_pressure": 123.2,
    "tubing_pressure": 118.9,
    "line_pressure": 105.9,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 5
  },
  {
    "timestamp": "2025-07-18T16:30:00Z",
    "casing_pressure": 121.5,
    "tubing_pressure": 113.8,
    "line_pressure": 98.3,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 5
  },
  {
    "timestamp": "2025-07-18T16:31:00Z",
    "casing_pressure": 129.4,
    "tubing_pressure": 122.7,
    "line_pressure": 97.3,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 5
  },
  {
    "timestamp": "2025-07-18T16:32:00Z",
    "casing_pressure": 127.5,
    "tubing_pressure": 124.8,
    "line_pressure": 97.3,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 5
  },
  {
    "timestamp": "2025-07-18T16:33:00Z",
    "casing_pressure": 124.7,
    "tubing_pressure": 125.7,
    "line_pressure": 102.3,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 5
  },
  {
    "timestamp": "2025-07-18T16:34:00Z",
    "casing_pressure": 123.3,
    "tubing_pressure": 121.3,
    "line_pressure": 97.3,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 5
  },
  {
    "timestamp": "2025-07-18T16:35:00Z",
    "casing_pressure": 120.3,
    "tubing_pressure": 119.9,
    "line_pressure": 97.4,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 5
  },
  {
    "timestamp": "2025-07-18T16:36:00Z",
    "casing_pressure": 122.0,
    "tubing_pressure": 116.9,
    "line_pressure": 102.0,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 5
  },
  {
    "timestamp": "2025-07-18T16:37:00Z",
    "casing_pressure": 127.9,
    "tubing_pressure": 125.8,
    "line_pressure": 99.8,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 5
  },
  {
    "timestamp": "2025-07-18T16:38:00Z",
    "casing_pressure": 121.0,
    "tubing_pressure": 115.6,
    "line_pressure": 103.4,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 5
  },
  {
    "timestamp": "2025-07-18T16:39:00Z",
    "casing_pressure": 126.9,
    "tubing_pressure": 129.3,
    "line_pressure": 107.8,
    "non_arrival_count": 0,
    "well_id": "WELL-001",
    "cycle_id": 5
  }
]


Query = Can you detect any anomalies in the data? If so, please describe them.
"""